# ChatAgent 聊天示例

这是一个基于 ChatAgent 的简单聊天机器人示例，支持流式和非流式两种模式。


In [2]:
project_root

WindowsPath('E:/Python_Project/OmniPok-Agent')

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# 添加项目根目录到路径
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 加载环境变量
env_path = project_root / ".env"
load_dotenv(env_path)

from src.agent.core import RunContext, InMemoryMemory, OmniPokLLM
from src.agent.agents import ChatAgent


## 初始化 ChatAgent


In [2]:
# 创建LLM实例（自动检测环境变量）
llm = OmniPokLLM()

# 创建ChatAgent
chat_agent = ChatAgent(
    agent_id="chat-bot-1",
    llm=llm,
    memory=InMemoryMemory(),
    system_prompt="你是一个友好、幽默的AI助手，喜欢用中文聊天。"
)

# 创建运行上下文
context = RunContext(
    tenant_id="user-1",
    user_id="user-1",
    budget=None,
    max_steps=10
)

print("✅ ChatAgent 初始化完成！")


✅ ChatAgent 初始化完成！


## 示例1: 非流式聊天（一次性返回）


In [3]:
# 非流式聊天 - 一次性返回完整响应
response = await chat_agent.chat("你好，请介绍一下你自己", context)
print(f"\n📝 完整响应: {response}")



📝 完整响应: 你好啊，我是一名友好、幽默的AI助手，可以陪你聊天、回答问题，或者提供一些有趣的知识。有什么想要了解或者聊一聊的吗？


## 示例2: 流式聊天（实时显示）


In [4]:
# 流式聊天 - 实时显示响应片段
response = await chat_agent.chat_stream("请用一句话介绍人工智能", context)
print(f"\n📝 完整响应: {response}")


🤖 AI: 🧠 正在调用 gpt-3.5-turbo 模型...
✅ 大语言模型响应成功:
人人工工智智能能是是利利用用计计算算机机科科学学模模拟拟人人类类智智能能的的领领域域，，旨旨在在让让机机器器具具备备学学习习、、推推理理、、理理解解语语言言等等能能力力。。


📝 完整响应: 人工智能是利用计算机科学模拟人类智能的领域，旨在让机器具备学习、推理、理解语言等能力。


## 示例3: 多轮对话（保持上下文）


In [11]:
response = await chat_agent.chat("This is a new message", context)
print(f"\n📝 完整响应: {response}")


📝 完整响应: Got it! Feel free to share anything you'd like to talk about or ask me. I'm here to chat with you!


In [12]:
chat_agent.memory.get_messages(chat_agent.agent_id)


[Message(role=<MessageRole.USER: 'user'>, content='你好，请介绍一下你自己', name=None, tool_calls=None, tool_call_id=None, timestamp=datetime.datetime(2025, 12, 28, 1, 7, 4, 382079)),
 Message(role=<MessageRole.ASSISTANT: 'assistant'>, content='你好啊，我是一名友好、幽默的AI助手，可以陪你聊天、回答问题，或者提供一些有趣的知识。有什么想要了解或者聊一聊的吗？', name=None, tool_calls=None, tool_call_id=None, timestamp=datetime.datetime(2025, 12, 28, 1, 7, 7, 233644)),
 Message(role=<MessageRole.USER: 'user'>, content='请用一句话介绍人工智能', name=None, tool_calls=None, tool_call_id=None, timestamp=datetime.datetime(2025, 12, 28, 1, 7, 13, 34063)),
 Message(role=<MessageRole.USER: 'user'>, content='请用一句话介绍人工智能', name=None, tool_calls=None, tool_call_id=None, timestamp=datetime.datetime(2025, 12, 28, 1, 7, 13, 34063)),
 Message(role=<MessageRole.ASSISTANT: 'assistant'>, content='人工智能是利用计算机科学模拟人类智能的领域，旨在让机器具备学习、推理、理解语言等能力。', name=None, tool_calls=None, tool_call_id=None, timestamp=datetime.datetime(2025, 12, 28, 1, 7, 14, 961556)),
 Message(role=<MessageRole.ASSISTAN

In [5]:
# 第一轮对话
response1 = await chat_agent.chat("我的名字是张三", context)
print(f"\n第一轮: {response1}")

# 第二轮对话（应该能记住名字）
response2 = await chat_agent.chat("你还记得我的名字吗？", context)
print(f"\n第二轮: {response2}")



第一轮: 很高兴认识你，张三！有什么问题或者想要聊的事情吗？欢迎随时和我交流哦。

第二轮: 当然记得，你的名字是张三。有什么我可以帮到你的吗，张三？


## 示例4: 直接使用 LLM 接口（流式 vs 非流式）


In [ ]:
# 方式1: 非流式调用（一次性返回）
messages = [
    {"role": "user", "content": "用一句话介绍Python"}
]

print("\n=== 非流式调用 ===")
response = llm.invoke(messages)
print(f"响应: {response}")


In [ ]:
# 方式2: 流式调用（实时显示）
messages = [
    {"role": "user", "content": "用一句话介绍机器学习"}
]

print("\n=== 流式调用 ===")
print("响应: ", end="", flush=True)
full_response = ""
for chunk in llm.think(messages):
    print(chunk, end="", flush=True)
    full_response += chunk
print()
print(f"\n完整响应: {full_response}")


In [ ]:

from dotenv import load_dotenv
from src.agent import BaseAgent, InMemoryMemory, global_registry
from src.agent.agents.text_agent import text_agent
from src.agent.core.llm import OmniPokLLM
llm = OmniPokLLM()

agent = TextAgent(
    name="AI助手",
    llm=llm,
    system_prompt="你是一个有用的AI助手"
)